# Segmenting and Clustering Neighborhoods in Toronto

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Webscrapping

Get url and create a soup object.

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text 
soup = BeautifulSoup(data, 'html5lib')

Locate the table of interest and put the postal code, borough and neighborhood information into separate lists.

In [5]:
tables = soup.find_all("tbody") #find tables
table = tables[0] #select table of interest

table_rows = table.find_all("tr") #select table rows

post_codes = [] #empty list to insert post codes
boroughs_and_neighborhoods = []

for i,row in enumerate(table_rows):
    cells = row.find_all("td") 
    for j,cell in enumerate(cells):
        if cell.find_all("i") == []: #discard the "Not assigned" cells
            
            post_code = cell.p.b.string #post codes
            borough_and_neighborhood = cell.p.span.text #borough and neighborhoods
            
            post_codes.append(post_code) #append post codes to list named post_codes
            boroughs_and_neighborhoods.append(borough_and_neighborhood) #append borough and neighborhoods to list named neighborhoods


# Create separate lists containing borough and neighboorhoods data accordingly: 
#Split the list named neighborhoods and replace some unwanted characters
borough = []
neighborhood = []

for i in range(len(boroughs_and_neighborhoods)):
    boroughs_and_neighborhoods[i] = boroughs_and_neighborhoods[i].strip(")").replace(")"," ").strip(" ").replace("/",",").replace(" , ", ", ").split("(")
    neighborhood.append(boroughs_and_neighborhoods[i][1])
    borough.append(boroughs_and_neighborhoods[i][0])

Use the lists containing the postal codes, boroughs and neighborhoods to create a dataframe using Pandas. 

In [6]:
data = {"PostalCode" : post_codes,
        "Borough" : borough,
        "Neighborhood": neighborhood}

df = pd.DataFrame(data)

# Clean some Borough Data and assign the neighborhood as the borough when not assigned
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
#Print the number of rows of the dataframe
df.shape

(103, 3)

## Geospatial coordinates

Read the Geospatial Coordinates csv file. 

In [8]:
df_gc = pd.read_csv('Geospatial_Coordinates.csv')
df_gc

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Order the neighborhood dataframe in same way as the Geospatial dataframe and reorder the index. Then append the latitude and longitude columns to the neighborhood dataframe.

In [9]:
# Sort values by postal code and set reorder the index
df.sort_values(by="PostalCode", inplace=True)
df.set_index(pd.Index([*range(0,len(df))]), inplace=True)

# Append latitude and Longitude Columns
df["Latitude"] = df_gc["Latitude"] 
df["Longitude"] = df_gc["Longitude"]
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


## Clustering of Toronto neighborhoods

Create a map of New York with neighborhoods superimposed on top.

In [10]:
import folium

# Create map of Toronto using latitude and longitude values

latitude =43.6511
longitude= -79.38
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Retrieve Toronto's venues.

In [11]:
nearby_venues1 = pd.read_json("https://raw.githubusercontent.com/ibm-developer-skills-network/yczvh-DataFilesForIBMProjects/master/segmenting_neighborhoods.json")    
nearby_venues1.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                 'Venue', 
                 'Venue Latitude', 
                 'Venue Longitude', 
                 'Venue Category']
Toronto_venues=nearby_venues1
Toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
...,...,...,...,...,...,...,...
1332,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,LCBO,43.741508,-79.584501,Liquor Store
1333,"Clairville, Humberwood, Woodbine Downs, West H...",43.706748,-79.594054,Economy Rent A Car,43.708471,-79.589943,Rental Car Location
1334,"Clairville, Humberwood, Woodbine Downs, West H...",43.706748,-79.594054,Logistics Distribution,43.707554,-79.589252,Bar
1335,"Clairville, Humberwood, Woodbine Downs, West H...",43.706748,-79.594054,Saand Rexdale,43.705072,-79.598725,Drugstore


Let's analyze the Downtown Toronto borough and retrieve its venues.

In [12]:
downtown_toronto = df[df['Borough']=='Downtown Toronto']
downtown_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
54,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
58,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
59,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


In [13]:
#take venues for downtown toronto neighborhoods
downtown_toronto_venues = Toronto_venues[Toronto_venues["Neighborhood"].isin(downtown_toronto["Neighborhood"])]
downtown_toronto_venues.set_index(pd.Index([*range(0,len(downtown_toronto_venues))]), inplace=True)
downtown_toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
...,...,...,...,...,...,...,...
451,Christie,43.669542,-79.422564,Marlenes Just Babies,43.671824,-79.420499,Baby Store
452,Christie,43.669542,-79.422564,Dupont Disco,43.670490,-79.426611,Nightclub
453,Christie,43.669542,-79.422564,Marian Engel Park,43.673754,-79.423988,Park
454,Christie,43.669542,-79.422564,Garrison Creek Park,43.671690,-79.427805,Park


Let's analyze Downtown Toronto venues in the different neighborhoods.

In [14]:
# one hot encoding
DowntownToronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DowntownToronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DowntownToronto_onehot.columns[-1]] + list(DowntownToronto_onehot.columns[:-1])
DowntownToronto_onehot = DowntownToronto_onehot[fixed_columns]

DowntownToronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category 

In [15]:
downtown_toronto_grouped = DowntownToronto_onehot.groupby('Neighborhood').mean().reset_index()
downtown_toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.033333,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.033333
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.033333
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.033333,0.066667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.033333,0.000000,0.000000,0.000000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.033333,0.00,0.000000,0.033333,0.000000,0.066667,0.033333


Now, let's return the 10 most common venues in Downtown Toronto.

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Farmers Market,Cocktail Bar,Beer Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Museum,Restaurant,Bistro,Liquor Store
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Rental Car Location,Boat or Ferry,Plane,Harbor / Marina,Coffee Shop,Airport Gate,Airport Food Court
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Yoga Studio,Middle Eastern Restaurant,Japanese Restaurant,Department Store,Sandwich Place,Comic Shop,Ramen Restaurant
3,Christie,Grocery Store,Café,Park,Nightclub,Coffee Shop,Baby Store,Restaurant,Italian Restaurant,Candy Store,Comfort Food Restaurant
4,Church and Wellesley,Coffee Shop,Pub,Japanese Restaurant,Dance Studio,Beer Bar,Salon / Barbershop,Creperie,Restaurant,Ramen Restaurant,Bookstore


Run _k_-means to cluster the neighborhood into 7 clusters.

In [17]:
# set number of clusters
kclusters = 7

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0, n_init=1000).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 5, 2, 1, 3, 3, 6, 1, 6])

Let's create a new dataframe with the 10 most common venues and the cluster labels.

In [18]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_toronto_merged = downtown_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [19]:
downtown_toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Comic Shop,Department Store,Deli / Bodega,Dance Studio,Creperie,Cosmetics Shop,Concert Hall
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,1,Coffee Shop,Restaurant,Italian Restaurant,Café,Bakery,Diner,Jewelry Store,Beer Store,Deli / Bodega,Liquor Store
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Pub,Japanese Restaurant,Dance Studio,Beer Bar,Salon / Barbershop,Creperie,Restaurant,Ramen Restaurant,Bookstore
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,5,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Gym / Fitness Center,Greek Restaurant,Pub,Performing Arts Venue,Mexican Restaurant
54,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,6,Café,Clothing Store,Theater,Burrito Place,Mexican Restaurant,Shopping Mall,Sandwich Place,Comic Shop,Ramen Restaurant,Bookstore


Finally, visualize the clusters.

In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examining the clusters one by one.

In [23]:
# Cluster 1
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[2] + list(range(6, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Rosedale,Park,Playground,Trail,Comic Shop,Department Store,Deli / Bodega,Dance Studio,Creperie,Cosmetics Shop,Concert Hall


In [24]:
# Cluster 2
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[2] + list(range(6, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,"St. James Town, Cabbagetown",Coffee Shop,Restaurant,Italian Restaurant,Café,Bakery,Diner,Jewelry Store,Beer Store,Deli / Bodega,Liquor Store
52,Church and Wellesley,Coffee Shop,Pub,Japanese Restaurant,Dance Studio,Beer Bar,Salon / Barbershop,Creperie,Restaurant,Ramen Restaurant,Bookstore
55,St. James Town,Gastropub,Café,Coffee Shop,Farmers Market,BBQ Joint,Cocktail Bar,New American Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Creperie
56,Berczy Park,Seafood Restaurant,Farmers Market,Cocktail Bar,Beer Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Museum,Restaurant,Bistro,Liquor Store
59,"Harbourfront East, Union Station, Toronto Islands",Café,Hotel,Park,Plaza,Basketball Stadium,Italian Restaurant,Skating Rink,Japanese Restaurant,New American Restaurant,IT Services


In [25]:
# Cluster 3
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[2] + list(range(6, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Christie,Grocery Store,Café,Park,Nightclub,Coffee Shop,Baby Store,Restaurant,Italian Restaurant,Candy Store,Comfort Food Restaurant


In [26]:
# Cluster 4
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[2] + list(range(6, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,"Richmond, Adelaide, King",Coffee Shop,Café,Seafood Restaurant,Steakhouse,Monument / Landmark,Pizza Place,Plaza,Colombian Restaurant,Lounge,Concert Hall
60,"Toronto Dominion Centre, Design Exchange",Café,Coffee Shop,Restaurant,Bakery,Japanese Restaurant,Wine Bar,Sandwich Place,American Restaurant,Art Gallery,Beer Bar
61,"Commerce Court, Victoria Hotel",Café,Hotel,American Restaurant,Restaurant,Coffee Shop,Gastropub,Wine Bar,Pub,New American Restaurant,Museum
70,"First Canadian Place, Underground city",Café,Restaurant,Coffee Shop,Seafood Restaurant,Hotel,Sandwich Place,Pub,Pizza Place,Japanese Restaurant,Wine Bar


In [27]:
# Cluster 5
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[2] + list(range(6, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Rental Car Location,Boat or Ferry,Plane,Harbor / Marina,Coffee Shop,Airport Gate,Airport Food Court


In [28]:
# Cluster 6
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 5, downtown_toronto_merged.columns[[2] + list(range(6, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Breakfast Spot,Café,Gym / Fitness Center,Greek Restaurant,Pub,Performing Arts Venue,Mexican Restaurant
57,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Yoga Studio,Middle Eastern Restaurant,Japanese Restaurant,Department Store,Sandwich Place,Comic Shop,Ramen Restaurant


In [29]:
# Cluster 7
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 6, downtown_toronto_merged.columns[[2] + list(range(6, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,"Garden District, Ryerson",Café,Clothing Store,Theater,Burrito Place,Mexican Restaurant,Shopping Mall,Sandwich Place,Comic Shop,Ramen Restaurant,Bookstore
66,"University of Toronto, Harbord",Café,Bar,Italian Restaurant,Japanese Restaurant,Bakery,Bookstore,Poutine Place,Beer Bar,Sandwich Place,Restaurant
67,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Mexican Restaurant,Wine Bar,Organic Grocery,Belgian Restaurant,Record Shop,Caribbean Restaurant,Pizza Place,Cocktail Bar
